## PyTorch - Pre Trained Models

In [1]:
import torchvision.models as models
import torch
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
from torchvision import transforms
from PIL import Image
from pathlib import Path
import numpy as np
from torch.autograd import Variable
import requests
import os
import json
import pprint
import pandas as pd
import json
import image_preprocessing_library as lib
import warnings
warnings.filterwarnings('ignore')

C:\ProgramData\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


### Model Dictionary

In [2]:
model_dict = {
#     "resnet18" : models.resnet18(pretrained=True),
#     "alexnet" : models.alexnet(pretrained=True),
#     "squeezenet" : models.squeezenet1_0(pretrained=True),
#     "vgg16" : models.vgg16(pretrained=True),
#     "densenet161" : models.densenet161(pretrained=True),
#     "googlenet" : models.googlenet(pretrained=True),
#     "shufflenet_v2" : models.shufflenet_v2_x1_0(pretrained=True),
#     "mobilenet_v2" : models.mobilenet_v2(pretrained=True),
#     "resnext50_32x4d" : models.resnext50_32x4d(pretrained=True),
    "wide_resnet50_2" : models.wide_resnet50_2(pretrained=True),
    "mnasnet" : models.mnasnet1_0(pretrained=True),
    "inception_v3" : models.inception_v3(pretrained=True)
}

### Pre-Processing Sequence


In [3]:
pre_processing_seq_dict = {
    "seq_0" : [], # for raw seq
    "seq_1" : ["gray"],
    "seq_2" : ["hsv"],
    "seq_3" : ["sharpen"],
    "seq_4" : ["gray", "bilateral_blur", "threshold_mean"],
    "seq_5" : ["gray", "bilateral_blur", "threshold_gaussian"],
    "seq_6" : ["gray", "bilateral_blur", "threshold_otsu"],
    "seq_7" : ["median_blur"],
    "seq_8" : ["gaussian_blur"],
    "seq_9" : ["bilateral_blur"],
    "seq_10" : ["fastnl_blur"],
    "seq_11" : ["gray", "bilateral_blur", "threshold_otsu", "opening"],
    "seq_12" : ["gray", "bilateral_blur", "threshold_otsu", "closing"],
    "seq_13" : ["opening"],
    "seq_14" : ["closing"],
    "seq_15" : ["gray", "sobel"],
    "seq_16" : ["gray", "laplacian"],
    "seq_17" : ["gray", "canny"]
}

### Config/Map files load

In [4]:
# dictionary containing imagenet - custom label maps
imagenet_label_map = None
with open('./config_jsons/imagenet_label_map.json') as json_file: 
    imagenet_label_map = json.load(json_file)

# dictionary containing model ids - model_name map
model_ids = None
with open('./config_jsons/model_ids_map.json') as json_file: 
    model_ids = json.load(json_file)

# dictionary containing dataset_ids and dataset_desc    
dataset_ids = None
with open('../../dataset/image_classification/dataset_id_map.json') as json_file:
    dataset_ids = json.load(json_file)
    
custom_dataset_labels = None
with open('./config_jsons/label_ids_map.json') as json_file:
    custom_dataset_labels = json.load(json_file)

#### Change in approach, instead of generating n dataset, we will process it on memory, i.e process images while evaluating.
Consider making below cell parameterised later using papermill, or you may prefer to do it manually

In [8]:
# taking path as a parameter here, to ease out refactoring in future.
# first experimentation is on raw dataset, will scale out to others
dataset_path = "../../dataset/image_classification/raw"
master_df_path = "./experiment_results/final_results/master_pytorch.csv"

In [9]:
# constants and declarations
base_path = Path(dataset_path)
LABELS_URL = 'https://s3.amazonaws.com/outcome-blog/imagenet/labels.json'
response = requests.get(LABELS_URL)  
labels = {int(key): value for key, value in response.json().items()}

In [7]:
# master dataframe initialise
# create a dataframe for storing per image predictions
# columns = ["model_id", "model_name", "seq_id", "seq_name", "image_name", "label_id", 
#             "pred_label_id", "label_name","pred_label_name", "pred_confidence"]
# temp = pd.DataFrame(columns = columns)
# temp.to_csv(master_df_path, index=False)

# above code is to create a master empty csv file with our columns

In [10]:
columns = ["model_id", "model_name", "seq_id", "seq_name", "image_name", "label_id", 
             "pred_label_id", "label_name","pred_label_name", "pred_confidence"]

# master_df = pd.read_csv('./experiment_results/master.csv')

# # creating a current run dataframe, for efficiency
# current_run_df = pd.DataFrame(columns = columns)

In [11]:
custom_labels = [x for x in os.walk(base_path)][0][1]

### Functions Definitions

#### OpenCV functions

In [12]:
def convert_to_pil_img(opencv_img):
    if opencv_img.dtype == 'float64':
        opencv_img = opencv_img.astype(np.uint8)
    pil_img = cv2.cvtColor(opencv_img, cv2.COLOR_BGR2RGB)
    pil_img = Image.fromarray(pil_img)
    return pil_img

def convert_to_cv_img(pil_img):
    np_img_arr = np.asarray(pil_img)
    cv_image=cv2.cvtColor(np_img_arr, cv2.COLOR_RGB2BGR)
    return cv_image

In [13]:
def get_model_id(model_name):
    for id, name in model_ids.items():
        if name == model_name:
            return id
    return "not found for model name: " + model_name

def get_model_name(id):
    if id not in model_ids.keys():
        return "not found for model id: " + str(id)
    return model_ids[id]

def get_seq_name(seq_id):
    if seq_id not in pre_processing_seq_dict.keys():
        return "not found for dataset id: " + seq_id
    return " > ".join(get_seq_operations(seq_id))

def get_label_id(label_name):
    for id, name in custom_dataset_labels.items():
        if name == label_name:
            return id
    return "not found for label: "+label_name

def get_label_name(id):
    if id not in custom_dataset_labels.keys():
        return "not found for id: " + str(id)
    return custom_dataset_labels[id]

def get_pred_label_id(pred_label_name):
    for custom_label, imagenet_labels in imagenet_label_map.items():
        if pred_label_name.lower() in imagenet_labels:
            return get_label_id(custom_label)
    return get_label_id("others")

def get_seq_operations(seq_id):
    return pre_processing_seq_dict[seq_id]

def load_master_df_from_csv():
    return pd.read_csv(master_df_path)

def dump_master_df_to_csv(master_df):
    master_df.to_csv(master_df_path, header=True, columns=columns, index=False)
    
def log_per_label_results_to_csv(model_name, label_name, pred_result_list, seq_id):
    # input - pred_result_list: (label, pred_tuple_list), pred_tuple_list: [(image_name, pred_label, pred_percentage)]
    # columns = ["model_id", "model_name", "seq_id", "seq_name", "image_name", "label_id", 
    #            "pred_label_id", "label_name","pred_label_name", "pred_confidence"]

    model_id = get_model_id(model_name)
    model_name = model_name
    # change below before running on pre processed image dataset
    seq_id = seq_id
    seq_name = get_seq_name(seq_id)
    label_id = get_label_id(label_name)
    label_name = label_name
    
    rows = []
    for res in pred_result_list:
        image_name = res[0]
        pred_label_name = res[1]
        pred_confidence = res[2]
        pred_label_id = get_pred_label_id(pred_label_name)
        row = [model_id, model_name, seq_id, seq_name, image_name, label_id, label_name, 
               pred_label_id, pred_label_name, pred_confidence]
        rows.append(row)
    
    current_label_df = pd.DataFrame(rows, columns = columns)
    master_df = load_master_df_from_csv()
    master_df = master_df.append(current_label_df)
    dump_master_df_to_csv(master_df)

In [14]:
# img_gen = next(load_per_label_imgs_generator())
# trans_img = apply_tranformations(img_gen[1][0][1], "seq_15")
# model = models.resnet18(pretrained=True)
# model.eval()
# out = model(trans_img)
# _, index = torch.max(out, 1)
# percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
# print("label is: " + labels[index[0].item()] + " and perc is: " + str(percentage[index[0]].item()))

In [15]:
# it works on list of images and return a list
def load_per_label_imgs_generator():  
    #better due to memory restrictions we read per class images at once i.e airplane folder at a time.
    # output: (label, [(image_name, pil_image)])
    
    # consider removing "others" label id from custom_dataset_label, while reading based on its values.
    for label in custom_labels:
        per_label_images = []
        label_image_names = [x for x in os.walk(base_path/label)][0][2]
        for image_name in label_image_names:
            img = Image.open(base_path/label/image_name)
            #TO-DO : pre run this step of converting to RGB and remove from here
            rgb_im = img.convert('RGB')
            per_label_images.append((image_name, rgb_im))
        # change below line to read all images, currently it will read only 1 image per label
        yield (label, per_label_images)

    
def apply_cv_transformations(seq_id, pil_img):
    cv_img = convert_to_cv_img(pil_img)
    operations = get_seq_operations(seq_id)
    processed_img = cv_img
    for operation in operations:
        processed_img = lib.dispatcher[operation](processed_img)
    return convert_to_pil_img(processed_img)


def apply_tranformations(img, seq_id):
    isGrayImgs = "gray" in get_seq_operations(seq_id)
    # transform dict for color and gray
    transform_dict = {
        "resize" : transforms.Compose([            
            transforms.Resize(256),                    
            transforms.CenterCrop(224)               
        ]),
        "color" : transforms.Compose([              
            transforms.ToTensor(),                     
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ]),
        "gray" : transforms.Compose([              
            transforms.ToTensor()                
        ])
    }
    # resizing using torch transform
    resized_img = transform_dict["resize"](img)

    # applying cv transforms
    transformed_img = apply_cv_transformations(seq_id, resized_img)

    # choosing torch tranform
    if isGrayImgs :
        transformed_img = transform_dict["gray"](transformed_img)
    else:
        transformed_img = transform_dict["color"](transformed_img)

    #convert it in to format(batch_size, channel, height, width)
    transformed_img = transformed_img.unsqueeze(0)
    return Variable(transformed_img)

# works on list of images and returns a list
def transform_images(imgs, seq_id):    
    #input: imgs is [(image_name, pil_image)]
    #output: [(imagename, pil_transformed_image)]
    results = []
    
    for img_tuple in imgs:
        results.append((img_tuple[0], apply_tranformations(img_tuple[1], seq_id)))
    return results

In [16]:
def evaluate_results(model, batch_imgs):  
    #input: model is pytorch model, batch_imgs is [(image_name, pil_transformed_image)]
    #output: results is [(image_name, pred_label, pred_percentage)]
    
    results = []
    for batch_img in batch_imgs:  
        model.eval()
        out = model(batch_img[1])
        _, index = torch.max(out, 1)
        percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
        results.append((batch_img[0], labels[index[0].item()], percentage[index[0]].item()))
    return results



def run_per_seq_model_inference(model, model_name, seq_id):
    # input model: any pytorch pre trained model
    # per_label_images_tuple: currently it assumers per label images, and of the form (label, [label_images])
    # output: (label, pred_tuple_list), pred_tuple_list: [(image_name, pred_label, pred_percentage)]
    
    # load images here, and take pre-processing sequence as an input
    for per_label_images_tuple in load_per_label_imgs_generator():
        tranformed_imgs = transform_images(per_label_images_tuple[1], seq_id)
        # logging to csv is done inside below function, hence need to pass parameters
        pred_res = evaluate_results(model, tranformed_imgs) 
        label_name = per_label_images_tuple[0]
        log_per_label_results_to_csv(model_name, label_name, pred_res, seq_id)

    
def run_all_model_inference(model_dict):
    for model_name, model in model_dict.items():
        print("Started inference for: "+model_name)
        for seq_id in pre_processing_seq_dict.keys():
            print("inference started on: " + str(seq_id) + ", ", end='')
            run_per_seq_model_inference(model, model_name, seq_id)
            print("______ completed!")
        print("completed inference -----------------------------------")

### Model Evaluation

In [17]:
run_all_model_inference(model_dict)

Started inference for: wide_resnet50_2
inference started on: seq_0, ______ completed!
inference started on: seq_1, ______ completed!
inference started on: seq_2, ______ completed!
inference started on: seq_3, ______ completed!
inference started on: seq_4, ______ completed!
inference started on: seq_5, ______ completed!
inference started on: seq_6, ______ completed!
inference started on: seq_7, ______ completed!
inference started on: seq_8, ______ completed!
inference started on: seq_9, ______ completed!
inference started on: seq_10, ______ completed!
inference started on: seq_11, ______ completed!
inference started on: seq_12, ______ completed!
inference started on: seq_13, ______ completed!
inference started on: seq_14, ______ completed!
inference started on: seq_15, ______ completed!
inference started on: seq_16, ______ completed!
inference started on: seq_17, ______ completed!
completed inference -----------------------------------
Started inference for: mnasnet
inference started on